In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import sklearn
import os

# load data

In [3]:
path=r"C:\work\freelancing\plants-collage-project\PlantVillage-Dataset\raw\color"

In [7]:
images=[]
target=dict()
y=[]

In [10]:
for i,x in enumerate(os.listdir(path)):
    target[i]=x
    c=0
    for img in os.listdir(os.path.join(path,x)):
        images.append(plt.imread(os.path.join(os.path.join(path,x),img)))
        y.append(i)
        if c==100:
            break
        c+=1

In [22]:
X=np.array(images)
y=np.array(y)
X.shape,len(y)

((3838, 256, 256, 3), 3838)

In [34]:
from keras.utils import to_categorical
y=to_categorical(y)
y.shape

(3838, 38)

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3070, 256, 256, 3), (768, 256, 256, 3), (3070, 38), (768, 38))

In [25]:
target

{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry_(including_sour)___healthy',
 6: 'Cherry_(including_sour)___Powdery_mildew',
 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn_(maize)___Common_rust_',
 9: 'Corn_(maize)___healthy',
 10: 'Corn_(maize)___Northern_Leaf_Blight',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___healthy',
 14: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___healthy',
 22: 'Potato___Late_blight',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___healthy',
 27: 'Strawberry___Leaf_scorch',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: '

# model

In [30]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from keras.utils import to_categorical
from keras.models import Sequential

In [31]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=3,input_shape=(256, 256, 3)))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(len(target),activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                19612902  
Total params: 19,613,798
Trainable params: 19,613,798
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [36]:
model.fit(X_train,y_train,epochs=10)


Epoch 1/10
3070/3070 [==============================] - 174s 57ms/step - loss: 6704.6779 - accuracy: 0.2466
Epoch 2/10
3070/3070 [==============================] - 173s 56ms/step - loss: 250.0728 - accuracy: 0.6518
Epoch 3/10
3070/3070 [==============================] - 174s 57ms/step - loss: 121.0164 - accuracy: 0.7818
Epoch 4/10
3070/3070 [==============================] - 172s 56ms/step - loss: 95.2926 - accuracy: 0.8218
Epoch 5/10
3070/3070 [==============================] - 173s 56ms/step - loss: 61.7660 - accuracy: 0.8801
Epoch 6/10
3070/3070 [==============================] - 173s 56ms/step - loss: 39.9992 - accuracy: 0.9114
Epoch 7/10
3070/3070 [==============================] - 172s 56ms/step - loss: 35.3128 - accuracy: 0.9215
Epoch 8/10
3070/3070 [==============================] - 173s 56ms/step - loss: 60.3795 - accuracy: 0.8964
Epoch 9/10
3070/3070 [==============================] - 175s 57ms/step - loss: 40.5965 - accuracy: 0.9248
Epoch 10/10
3070/3070 [==================

# Testing

In [37]:
model.evaluate(X_test,y_test)

768/768 [==============================] - 14s 18ms/step


[588.1274757385254, 0.4739583432674408]

In [38]:
model.save("model-v1.h5")

# load model

In [11]:
from keras.models import load_model
model=load_model(r"C:\work\freelancing\plants-collage-project\models\model-v1.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                19612902  
Total params: 19,613,798
Trainable params: 19,613,798
Non-trainable params: 0
_________________________________________________________________


# save to .pb
# https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc

In [5]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [7]:
from keras import backend as K
import tensorflow as tf
# Create, compile and train model...

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 23 variables.
INFO:tensorflow:Converted 23 variables to const ops.


In [8]:
help(tf.train.write_graph)

Help on function write_graph in module tensorflow.python.framework.graph_io:

write_graph(graph_or_graph_def, logdir, name, as_text=True)
    Writes a graph proto to a file.
    
    The graph is written as a text proto unless `as_text` is `False`.
    
    ```python
    v = tf.Variable(0, name='my_variable')
    sess = tf.compat.v1.Session()
    tf.io.write_graph(sess.graph_def, '/tmp/my-model', 'train.pbtxt')
    ```
    
    or
    
    ```python
    v = tf.Variable(0, name='my_variable')
    sess = tf.compat.v1.Session()
    tf.io.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt')
    ```
    
    Args:
      graph_or_graph_def: A `Graph` or a `GraphDef` protocol buffer.
      logdir: Directory where to write the graph. This can refer to remote
        filesystems, such as Google Cloud Storage (GCS).
      name: Filename for the graph.
      as_text: If `True`, writes the graph as an ASCII proto.
    
    Returns:
      The path of the output proto file.



In [10]:
tf.train.write_graph(frozen_graph, "", "my_model.pb", as_text=False)

'my_model.pb'

# save to TFllite

In [13]:
#from_keras_model_file
file=r"C:\work\freelancing\plants-collage-project\models\model-v1.h5"
converter = tf.lite.TFLiteConverter.from_keras_model_file(file)
# Convert the model
tflite_model = converter.convert()
# Create the tflite model file
tflite_model_name = "mymodel.tflite"
open(tflite_model_name, "wb").write(tflite_model)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


78456504